In [2]:
import tkinter as tk
from tkinter import messagebox
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors

# Spotify API credentials
client_id = ''
client_secret = ''

# Authenticate with Spotify
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id, client_secret=client_secret))

# Load the dataset
df = pd.read_csv('spotify_tracks.csv')

# Select features
features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']
X = df[features]

# Normalize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Fit the KNN model
knn = NearestNeighbors(n_neighbors=10, algorithm='auto').fit(X_scaled)  # Increased neighbors to filter unique artists later

# Function to get track features using band and song name
def get_track_features(artist_name, track_name):
    results = sp.search(q=f'artist:{artist_name} track:{track_name}', type='track', limit=1)
    if results['tracks']['items']:
        track = results['tracks']['items'][0]
        track_id = track['id']
        track_features = sp.audio_features(track_id)[0]
        return track_features
    else:
        return None

# Function to find nearest neighbors
def find_nearest_neighbors(track_features):
    track_df = pd.DataFrame([track_features])
    track_scaled = scaler.transform(track_df[features])
    distances, indices = knn.kneighbors(track_scaled)
    neighbors = df.iloc[indices[0]]
    unique_artists = neighbors.drop_duplicates(subset='artists').head(5)  # Keep unique artists and limit to 5
    return unique_artists

# Function to handle the button click
def on_submit():
    artist_name = artist_entry.get()
    track_name = track_entry.get()
    track_features = get_track_features(artist_name, track_name)
    if track_features:
        nearest_neighbors = find_nearest_neighbors(track_features)
        display_neighbors(nearest_neighbors)
    else:
        messagebox.showerror("Error", "Track not found")

# Function to display nearest neighbors in a visually appealing way
def display_neighbors(neighbors):
    result_text.delete(1.0, tk.END)
    for index, row in neighbors.iterrows():
        result_text.insert(tk.END, f"Track: {row['track_name']}\n")
        result_text.insert(tk.END, f"Artist: {row['artists']}\n")
        result_text.insert(tk.END, f"Album: {row['album_name']}\n")
        result_text.insert(tk.END, f"Popularity: {row['popularity']}\n")
        result_text.insert(tk.END, f"Danceability: {row['danceability']}\n")
        result_text.insert(tk.END, f"Energy: {row['energy']}\n")
        result_text.insert(tk.END, f"Key: {row['key']}\n")
        result_text.insert(tk.END, f"Loudness: {row['loudness']}\n")
        result_text.insert(tk.END, f"Mode: {row['mode']}\n")
        result_text.insert(tk.END, f"Speechiness: {row['speechiness']}\n")
        result_text.insert(tk.END, f"Acousticness: {row['acousticness']}\n")
        result_text.insert(tk.END, f"Instrumentalness: {row['instrumentalness']}\n")
        result_text.insert(tk.END, f"Liveness: {row['liveness']}\n")
        result_text.insert(tk.END, f"Valence: {row['valence']}\n")
        result_text.insert(tk.END, f"Tempo: {row['tempo']}\n")
        result_text.insert(tk.END, "-"*50 + "\n")

# Create the GUI
root = tk.Tk()
root.title("Spotify Nearest Neighbors")

# Create and place the components
tk.Label(root, text="Artist Name").grid(row=0, column=0, padx=10, pady=10)
tk.Label(root, text="Track Name").grid(row=1, column=0, padx=10, pady=10)
artist_entry = tk.Entry(root)
track_entry = tk.Entry(root)
artist_entry.grid(row=0, column=1, padx=10, pady=10)
track_entry.grid(row=1, column=1, padx=10, pady=10)

submit_button = tk.Button(root, text="Find Nearest Neighbors", command=on_submit)
submit_button.grid(row=2, columnspan=2, pady=10)

result_text = tk.Text(root, height=20, width=80, wrap='word', padx=10, pady=10)
result_text.grid(row=3, columnspan=2, padx=10, pady=10)

# Run the GUI
root.mainloop()


SpotifyOauthError: No client_id. Pass it or set a SPOTIPY_CLIENT_ID environment variable.